# Scrapy

> #### Scrapy란
웹사이트에서 필요한 데이터를 추출하는 오픈소스 프레임워크.
- 가볍고, 빠르고, 확장성이 좋다.
    - python 기반으로 spider 코드 작성.

> #### Scrapy의 주요 특징
- 비동기 네트워킹 라이브러리(asynchronous networking library)인 Twisted를 기반
    - 매우 우수한 성능 발휘.
- XPath, CSS 표현식으로 HTML 소스에서 데이터 추출이 가능.
- webdriver를 사용하지 않음.
- 지정된 url만 조회하므로 셀레니움보다 가볍고 빠른 퍼포먼스를 냄.


cf. 셀레니움의 특징
- Xpath, CSS 표현식으로 HTML 소스에서 데이터 추출이 가능.
- webdriver를 사용.
- 페이지를 렌더링 하기 위해 필요한 js, css, image 파일까지 불러옴.

> #### Scrapy 예제 코드

In [ ]:
# quotes_spider.py
import scrapy
from scrapy.crawler import CrawlerProcess


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/page/1/',
            'http://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f'quotes-{page}.html'
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log(f'Saved file {filename}')

# 크롤러를 초기화하는 과정
if __name__ == '__main__':
	process = CrawlerProcess(setting)
    crawler = process.create_crawler(QuotesSpider)
    process.crawl(crawler)
    process.start()

> #### Scrapy 실행 코드

In [ ]:
scrapy crawl 'spider name'

> #### errback 코드 예제

In [ ]:
import scrapy

from scrapy.spidermiddlewares.httperror import HttpError
from twisted.internet.error import DNSLookupError
from twisted.internet.error import TimeoutError, TCPTimedOutError

class ErrbackSpider(scrapy.Spider):
    name = "errback_example"
    start_urls = [
        "http://www.httpbin.org/",              # HTTP 200 expected
        "http://www.httpbin.org/status/404",    # Not found error
        "http://www.httpbin.org/status/500",    # server issue
        "http://www.httpbin.org:12345/",        # non-responding host, timeout expected
        "http://www.httphttpbinbin.org/",       # DNS error expected
    ]

    # 호출이 성공하면 callback으로 선언된 parse_httpbin이 실행

    def start_requests(self):
        for u in self.start_urls:
            yield scrapy.Request(u, callback=self.parse_httpbin,
                                    errback=self.errback_httpbin,
                                    dont_filter=True)

    def parse_httpbin(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        # do something useful here...

    def errback_httpbin(self, failure):
        # log all failures
        self.logger.error(repr(failure))

        # 호출이 실패하면 errback으로 선언된 errback_httpbin이 호출

        if failure.check(HttpError):
            # these exceptions come from HttpError spider middleware
            # you can get the non-200 response
            response = failure.value.response
            self.logger.error('HttpError on %s', response.url)

        elif failure.check(DNSLookupError):
            # this is the original request
            request = failure.request
            self.logger.error('DNSLookupError on %s', request.url)

        elif failure.check(TimeoutError, TCPTimedOutError):
            request = failure.request
            self.logger.error('TimeoutError on %s', request.url)

#### Scrapy 특징
다양한 configuration 존재
- 데이터 다운로드 타임아웃 설정.
- 각 request간에 random한 텀(사람의 실제 액션처럼 보이기 위한)을 설정.

> #### Scrapy 장·단점
장점
 - 확장성이 좋음.
    - 미들웨어를 새로 개발하거나 파이프라인을 연결하는 게 쉬움.

단점
- javascript 지원이 어렵다.
    - ajax/pjax로 데이터가 갱신되는 웹페이지라면 원하는 데이터를 추출하는 게 쉽지 않음.

cf. 셀레니움 사용을 추천하는 경우
- 한 개 사이트 안에서 여러 페이지를 돌아다니며 핸들링해야 하는 데이터가 많을 때.
- 그 페이지가 javascript의 영향이 적을 때.